In [ ]:
# %pip install openai python-dotenv pandas

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import pandas as pd
from os import getenv

In [3]:
load_dotenv()

True

In [4]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_API_KEY"),
)

In [14]:
def generate_mcqs(context, assessment_los, num_questions=30):
    """
    Generate MCQs from a given context using an LLM.
    """
    prompt = f"""
    You are a master educator that is proficient in teaching students and 
    creatting multiple-choice questions (MCQs) assessment that assesses their 
    understanding of given topics and achieves the learning outcomes of the assessment.
    Generate {num_questions} MCQs based on the following context which includes a list of topics covered in a Python programming course, 
    and the learning outcomes of the assessment.
    Each question should have 4 options, with one correct answer and three distractors.
    Please ensure the questions are relevant, clear, and cover key aspects of the context.
    The questions should be in a variety of difficulty levels (i.e. easy, medium, hard).
    
    Context: {context}
    Assessment learning outcomes: {assessment_los}
    
    Format each question as follows:
    Q1. [Question text]
    A) [Option 1]
    B) [Option 2]
    C) [Option 3]
    D) [Option 4]
    Correct Answer: [Correct Option]
    Difficulty: [Difficulty Level]
    
    Please return only the generated MCQs in a list.
    
    """
    
    response = client.chat.completions.create(
        model="microsoft/phi-4",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant that generates multiple-choice questions."},
            {"role": "user", "content": prompt}
        ],
        # max_tokens=1000
    )
    
    mcqs = response.choices[0].message.content
    return mcqs

In [13]:
# Read the context from a text file 
context = ""
with open("content.txt", "r") as f:
    context = f.read()
    
print(context)

Topic 1: Introduction to Problem Solving, Programming and Software Dev Methodologies
What is Problem Solving?
•“Finding a solution to a difficult or complex question or situation; to engage in the actions or thoughts necessary to discover solutions to problems.”

Types of Problems
•There are lots of different types of problems we can solve -and the difficulty of each problem can change dramatically. For example, let's try solving some of these problems…
If I have $50 and purchase 3 boxes of chocolates on sale for $9 each, how much change will I receive?
How can we reverse climate change?
How can we ensure we get good results for all the courses we are studying?

Routine and Non-Routing Problems
•Routine (simple)problems are the kinds of problems have simple solutions and that occur regularly.
•Non-Routine(complicated &complex)problems are more difficult and often require devisingstrategies to solve.
•Then there are wicked problems…

Characteristics of Wicked Problems
•Difficult to defi

In [7]:
# Read the learning outcomes from a text file 
assessment_los = ""
with open("A1_los.txt", "r") as f:
    assessment_los = f.read()
    
print(assessment_los)

Knowledge
K1. Identify and use the correct syntax of a common programming language.
K2. Recall and use typical programming constructs to design and implement simple software solutions.
K3. Reproduce and adapt commonly used basic algorithms.
K4. Explain the importance of programming style concepts (documentation, mnemonic names, indentation).
Skills
S1. Utilise pseudocode and/or algorithms as a major program design technique.
S2. Write and implement a solution algorithm using basic programming constructs.



In [15]:
mcqs = generate_mcqs(context, assessment_los)

In [16]:
mcqs = mcqs.split("\n\n")
mcqs

['```markdown\n1. Q1. What is problem solving in the context of programming?\n   A) Designing software architecture\n   B) Finding solutions to complex or difficult questions or situations\n   C) Writing documentation for code\n   D) Establishing software development methodologies\n   Correct Answer: B\n   Difficulty: Easy',
 '2. Q2. What distinguishes routine problems from non-routine problems in programming?\n   A) Routine problems are more complex and require advanced algorithms.\n   B) Non-routine problems are simple and occur regularly.\n   C) Routine problems have simple solutions and occur regularly.\n   D) Non-routine problems can be easily solved with standard techniques.\n   Correct Answer: C\n   Difficulty: Easy',
 '3. Q3. Which of the following is not a characteristic of a wicked problem?\n   A) Difficult to define\n   B) Has many interdependencies\n   C) Stable resolution\n   D) No clear solution\n   Correct Answer: C\n   Difficulty: Medium',
 '4. Q4. In the Six-Step Probl

In [17]:
print(mcqs[1])

2. Q2. What distinguishes routine problems from non-routine problems in programming?
   A) Routine problems are more complex and require advanced algorithms.
   B) Non-routine problems are simple and occur regularly.
   C) Routine problems have simple solutions and occur regularly.
   D) Non-routine problems can be easily solved with standard techniques.
   Correct Answer: C
   Difficulty: Easy


In [18]:
def evaluate_mcqs(mcqs, context):
    results = []
    for question in mcqs:
        prompt = f"""
        Evaluate the following MCQ based on the provided context.
        Return your evaluation in valid JSON format with scores (0-1) and justifications.

        Context: {context}

        MCQ: {question}

        Required JSON format:
        {{
            "question": "<question_text>",
            "evaluations": {{
                "relevance_score": <0-1>,
                "relevance_justification": "<text>",
                "clarity_score": <0-1>,
                "clarity_justification": "<text>",
                "difficulty_score": <0-1>,
                "difficulty_justification": "<text>",
                "correctness_score": <0-1>,
                "correctness_justification": "<text>"
            }}
        }}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an MCQ evaluation assistant. Always respond with valid JSON in the exact format specified."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000
            )

            eval_data = json.loads(response.choices[0].message.content)
            results.append({
                'Question': eval_data['question'],
                'Relevance Score': eval_data['evaluations']['relevance_score'],
                'Relevance Justification': eval_data['evaluations']['relevance_justification'],
                'Clarity Score': eval_data['evaluations']['clarity_score'],
                'Clarity Justification': eval_data['evaluations']['clarity_justification'],
                'Difficulty Score': eval_data['evaluations']['difficulty_score'],
                'Difficulty Justification': eval_data['evaluations']['difficulty_justification'],
                'Correctness Score': eval_data['evaluations']['correctness_score'],
                'Correctness Justification': eval_data['evaluations']['correctness_justification']
            })
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for question: {question}\nError: {str(e)}")
            continue

    return pd.DataFrame(results)

In [19]:
evaluate_mcqs(mcqs, context)

,Question,Relevance Score,Relevance Justification,Clarity Score,Clarity Justification,Difficulty Score,Difficulty Justification,Correctness Score,Correctness Justification
0,Q1. What is problem solving in the context of ...,1.00,The question is highly relevant to the context...,1.00,The question is clearly phrased and easy to un...,0.2,The question is relatively easy as it directly...,1.0,The provided answer 'B) Finding solutions to c...
1,2. Q2. What distinguishes routine problems fro...,1.00,The question is highly relevant to the problem...,1.00,The question is succinct and straightforward. ...,0.2,The question is easy as it directly asks about...,1.0,The correct answer (C) is accurately identifie...
2,Q3. Which of the following is not a characteri...,1.00,The question is highly relevant to the context...,1.00,"The question is clear and well-structured, mak...",0.5,The difficulty level of the question is medium...,1.0,The provided correct answer (C - 'Stable resol...
3,"Q4. In the Six-Step Problem Solving Process, w...",1.00,The question is highly relevant to the context...,1.00,The question is written in clear and simple la...,0.2,This question can be considered easy since the...,1.0,"The correct answer given, 'B) Analyze the prob..."
4,Q5. Which programming strategy involves trying...,1.00,The question is highly relevant to the provide...,1.00,The question is clear and easily understandabl...,0.5,The question is of a medium difficulty level a...,1.0,The designated correct answer 'trial and error...
5,Python is an imperative programming language. ...,1.00,The question is relevant to the context provid...,1.00,The question is clear and straightforward. It ...,0.5,The question is of medium difficulty as it req...,1.0,The correct answer (option C) is indeed correc...
6,Q7. Which of the following is not a typical me...,0.50,The provided context discusses problem solving...,1.00,"The question is clear and straightforward, ask...",0.8,The question is considered hard because it req...,1.0,The correct answer choice 'Quantum Execution' ...
7,Q8. What type of data does the 'int' type repr...,0.80,The question is relevant as it pertains to pro...,1.00,The question is clear and straightforward. It ...,0.4,The difficulty of the question is set to 'Easy...,1.0,The correct answer is indeed option C indicati...
8,"When dividing two integers in Python 3, what t...",0.60,The question is moderately relevant to the con...,1.00,The question is very clear and straightforward...,0.3,The difficulty of this question can be conside...,1.0,The provided correct answer (Float) is indeed ...
9,"10. Q10. What does a ""widening conversion"" in ...",0.00,The question is not relevant to the provided c...,1.00,The question is clear and straightforward. It ...,0.5,The question is of medium difficulty for someo...,1.0,The correct answer is indicated properly. Wide...
